# 地図作成のビジュアライゼーション

_「地図作成は、人類の最も長い歴史を持つ知的活動の一つであり、また最も複雑な活動の一つでもあります。科学的理論、グラフィカルな表現、地理的事実、実用的な考慮が絶え間ない形で融合しています。」_ — [H. J. スチュワード](https://books.google.com/books?id=cVy1Ms43fFYC)

地図作成（カートグラフィー）は、発見とデザインの何世紀にもわたる歴史を持つ分野です。地図作成ビジュアライゼーションは、位置、ルート、地球表面上の軌跡など、空間情報を含むデータを伝えるために地図作成技術を活用します。

<div style="float: right; margin-left: 1em; margin-top: 1em;"><img width="300px" src="https://gist.githubusercontent.com/jheer/c90d582ef5322582cf4960ec7689f6f6/raw/8dc92382a837ccc34c076f4ce7dd864e7893324a/latlon.png" /></div>

地球を球体として近似する場合、位置は_緯度_（赤道から北または南に何度かの角度）と_経度_（東西の位置を指定する角度）の球面座標系を使用して表すことができます。このシステムでは、_平行線_は一定の緯度を持つ円であり、_子午線_は一定の経度を持つ円です。[本初子午線](https://en.wikipedia.org/wiki/Prime_meridian)は0°経度に位置し、慣例としてイギリス・グリニッジの王立天文台を通過するものとして定義されています。

三次元の球体を二次元の平面に「平坦化」するためには、[投影法](https://en.wikipedia.org/wiki/Map_projection)を適用する必要があります。これにより、（経度、緯度）のペアを（x、y）の座標にマッピングします。[スケール](https://github.com/uwdata/visualization-curriculum/blob/master/altair_scales_axes_legends.ipynb)と似たように、投影法はデータのドメイン（空間的な位置）から視覚的なレンジ（ピクセル位置）にマッピングします。ただし、これまで見てきたスケールは一方向のドメインを受け入れていましたが、地図投影は本質的に二方向のものです。

このノートブックでは、Altairを使って地図を作成し、空間データを視覚化する基本を紹介します。内容は以下の通りです：

- 地理的特徴を表現するためのデータ形式、
- ポイント、シンボル、コロプレス地図などの地理的視覚化技法、
- 一般的な地図投影法のレビュー。

_このノートブックは[データ視覚化カリキュラム](https://github.com/uwdata/visualization-curriculum)の一部です。_

In [1]:
import pandas as pd
import altair as alt
from vega_datasets import data

## 地理データ：GeoJSON と TopoJSON

これまで、JSONやCSV形式のデータセットを使用して、行（レコード）と列（フィールド）からなるデータテーブルを操作してきました。地理的な地域（国、州、_など_）や軌跡（飛行ルート、地下鉄路線、_など_）を表現するには、豊富な幾何学的情報をサポートする追加の形式を導入する必要があります。

[GeoJSON](https://en.wikipedia.org/wiki/GeoJSON)は、特別なJSON形式内で地理的特徴をモデル化します。GeoJSONの`feature`は、国境を構成する`経度`、`緯度`座標などの幾何学的データと、その他のデータ属性を含むことができます。

以下は、アメリカ合衆国コロラド州の境界を示すGeoJSONの`feature`オブジェクトです:

~~~ json
{
  "type": "Feature",
  "id": 8,
  "properties": {"name": "Colorado"},
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [[-106.32056285448942,40.998675790862656],[-106.19134826714341,40.99813863734313],[-105.27607827344248,40.99813863734313],[-104.9422739227986,40.99813863734313],[-104.05212898774828,41.00136155846029],[-103.57475287338661,41.00189871197981],[-103.38093099236758,41.00189871197981],[-102.65589358559272,41.00189871197981],[-102.62000064466328,41.00189871197981],[-102.052892177978,41.00189871197981],[-102.052892177978,40.74889940428302],[-102.052892177978,40.69733266640851],[-102.052892177978,40.44003613055551],[-102.052892177978,40.3492571857556],[-102.052892177978,40.00333031918079],[-102.04930288388505,39.57414465707943],[-102.04930288388505,39.56823596836465],[-102.0457135897921,39.1331416175485],[-102.0457135897921,39.0466599009048],[-102.0457135897921,38.69751011321283],[-102.0457135897921,38.61478847120581],[-102.0457135897921,38.268861604631],[-102.0457135897921,38.262415762396685],[-102.04212429569915,37.738153927339205],[-102.04212429569915,37.64415206142214],[-102.04212429569915,37.38900413964724],[-102.04212429569915,36.99365914927603],[-103.00046581851544,37.00010499151034],[-103.08660887674611,37.00010499151034],[-104.00905745863294,36.99580776335414],[-105.15404227428235,36.995270609834606],[-105.2222388620483,36.995270609834606],[-105.7175614468747,36.99580776335414],[-106.00829426840322,36.995270609834606],[-106.47490250048605,36.99365914927603],[-107.4224761410235,37.00010499151034],[-107.48349414060355,37.00010499151034],[-108.38081766383978,36.99903068447129],[-109.04483707103458,36.99903068447129],[-109.04483707103458,37.484617466122884],[-109.04124777694163,37.88049961001363],[-109.04124777694163,38.15283644441336],[-109.05919424740635,38.49983761802722],[-109.05201565922046,39.36680339854235],[-109.05201565922046,39.49786885730673],[-109.05201565922046,39.66062637372313],[-109.05201565922046,40.22248895514744],[-109.05201565922046,40.653823231326896],[-109.05201565922046,41.000287251421234],[-107.91779872584989,41.00189871197981],[-107.3183866123281,41.00297301901887],[-106.85895696843116,41.00189871197981],[-106.32056285448942,40.998675790862656]]
    ]
  }
}
~~~

`feature`には`properties`オブジェクトが含まれており、ここには任意の数のデータフィールドが含まれます。さらに`geometry`オブジェクトがあり、今回は州の境界を示す`[経度, 緯度]`の座標からなる単一のポリゴンが含まれています。座標は右方向に少し進みますので、必要に応じてスクロールできます。

GeoJSONの詳細について学ぶには、[公式GeoJSON仕様書](http://geojson.org/)をご覧いただくか、[Tom MacWrightによる役立つ入門書](https://macwright.org/2015/03/23/geojson-second-bite)をお読みください。

GeoJSONのストレージ形式の欠点の一つは、冗長性があり、結果としてファイルサイズが大きくなることです。例えば、コロラド州は6つの他の州（アリゾナ州と接する角を含めると7つ）と境界を共有しています。それぞれの州について重複する座標リストを使用するのではなく、共有する境界を一度だけエンコードするという、よりコンパクトなアプローチが可能です。これにより、地理的な地域の_トポロジー_が表現されます。幸い、これこそが[TopoJSON](https://github.com/topojson/topojson/blob/master/README.md)フォーマットが行うことです！

世界の国々を110メートル解像度で示すTopoJSONファイルをロードしましょう:

In [2]:
world = data.world_110m.url
world

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/world-110m.json'

In [3]:
world_topo = data.world_110m()

In [4]:
world_topo.keys()

dict_keys(['type', 'transform', 'objects', 'arcs'])

In [5]:
world_topo['type']

'Topology'

In [6]:
world_topo['objects'].keys()

dict_keys(['land', 'countries'])

_上記の`world_topo` TopoJSON辞書オブジェクトを調べて、その内容を確認します。_

このデータでは、`objects`プロパティがデータから抽出できる名前付き要素を示しています：すべての`countries`のジオメトリ、または地球上のすべての`land`を表す単一のポリゴンです。これらのいずれかをGeoJSONデータに展開して視覚化することができます。

TopoJSONは専門的なフォーマットであるため、AltairにTopoJSONフォーマットを解析するように指示する必要があります。これにより、トポロジーから抽出したい名前付きの特徴オブジェクトを指定します。以下のコードは、`world`データセットから`countries`オブジェクトのGeoJSON特徴を抽出したいことを示しています:

~~~ js
alt.topo_feature(world, 'countries')
~~~

この`alt.topo_feature`メソッド呼び出しは、次のVega-Lite JSONに展開されます:

~~~ json
{
  "values": world,
  "format": {"type": "topojson", "feature": "countries"}
}
~~~

これで地理データをロードできるようになったので、地図作成を始めましょう！

## Geoshape Marks

地理データを視覚化するために、Altairは`geoshape`マークタイプを提供します。基本的な地図を作成するには、`geoshape`マークを作成し、TopoJSONデータを渡します。このデータは、各国のGeoJSON特徴に展開されます:

In [7]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape()

alt.Chart(...)

上記の例では、Altairはデフォルトで青色を適用し、デフォルトの地図投影法（`mercator`）を使用しています。標準のマークプロパティを使用して、色や境界の線幅をカスタマイズすることができます。`project`メソッドを使用することで、独自の地図投影法を追加することも可能です。

例えば、以下のコードでは、`project`メソッドを使って`albersUsa`投影法を適用し、国の境界線を太くし、色をカスタマイズしています:


In [8]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator'
)

alt.Chart(...)

デフォルトでは、Altairはデータ全体がチャートの幅と高さに収まるように投影を自動的に調整します。さらに、`scale`（ズームレベル）や`translate`（パン）などの投影パラメータを指定して、投影設定をカスタマイズすることができます。以下のコードでは、`scale`と`translate`パラメータを調整して、ヨーロッパに焦点を合わせています:


In [9]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator', scale=400, translate=[100, 550]
)

alt.Chart(...)

_注意点として、このスケールでデータの110m解像度が明らかになります。より詳細な海岸線や境界線を表示するには、より細かいジオメトリを持つ入力ファイルが必要です。`scale`と`translate`パラメータを調整して、地図を他の地域に焦点を合わせてください！_

これまでの地図は国だけを表示しています。`layer`オペレーターを使用することで、複数の地図要素を組み合わせることができます。Altairには、追加の地図レイヤーを作成するために使用できる_データ生成器_が含まれています：

- 球体生成器（`{'sphere': True}`）は、地球全体のGeoJSON表現を提供します。追加の`geoshape`マークを作成して、地球の形を背景レイヤーとして埋め込むことができます。
- 緯度経度線生成器（`{'graticule': ...}`）は、_緯線経線_を表すGeoJSONフィーチャーを作成します。デフォルトの緯線経線は、±80°の緯度ごとに10°ごとの経線と緯線を持っています。極地には、90°ごとの経線があります。これらの設定は、`stepMinor`および`stepMajor`プロパティを使用してカスタマイズできます。

球体、緯線経線、国のマークをレイヤーとして組み合わせた再利用可能な地図仕様を作成しましょう:

In [10]:
map = alt.layer(
    # use the sphere of the Earth as the base layer
    alt.Chart({'sphere': True}).mark_geoshape(
        fill='#e6f3ff'
    ),
    # add a graticule for geographic reference lines
    alt.Chart({'graticule': True}).mark_geoshape(
        stroke='#ffffff', strokeWidth=1
    ),
    # and then the countries of the world
    alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
        fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
    )
).properties(
    width=600,
    height=400
)

地図に希望する投影法を適用し、その結果を描画することができます。ここでは、[ナチュラル・アース投影](https://en.wikipedia.org/wiki/Natural_Earth_projection)を適用しています。_sphere_レイヤーは淡い青色の背景を提供し、_graticule_レイヤーは白い地理的参照線を提供します。

In [11]:
map.project(
    type='naturalEarth1', scale=110, translate=[300, 200]
).configure_view(stroke=None)

alt.LayerChart(...)

## ポイントマップ

GeoJSONやTopoJSONファイルで提供される_幾何学的_データに加えて、多くの表形式データセットには、`longitude`（経度）と`latitude`（緯度）の座標を示すフィールドや、国名、州名、郵便番号などの地理的地域への参照が含まれています。これらは[ジオコーディングサービス](https://en.wikipedia.org/wiki/Geocoding)を使用して座標にマッピングできます。場合によっては、位置データが十分に豊富で、データポイントのみを投影することで意味のあるパターンを見ることができます — 基本地図は必要ありません！

アメリカ合衆国の5桁の郵便番号のデータセットを見てみましょう。このデータセットには、各郵便局の`longitude`、`latitude`座標に加えて、`zip_code`フィールドも含まれています。

In [12]:
zipcodes = data.zipcodes.url
zipcodes

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/zipcodes.csv'

私たちは、各郵便局の位置を小さな（1ピクセル）`square`マークを使って可視化できます。ただし、位置を設定するために、`x`および`y`チャネルは使用しません。_なぜでしょうか？_

地図投影法は、（経度、緯度）座標を（x、y）座標にマッピングしますが、その方法は任意です。例えば、`longitude`が`x`に、`latitude`が`y`に対応している保証はありません！代わりに、Altairは特別な`longitude`および`latitude`エンコーディングチャネルを提供して、地理的座標を扱います。これらのチャネルは、どのデータフィールドを`longitude`および`latitude`座標にマッピングするかを示し、その後、座標を（x、y）位置にマッピングするための投影を適用します。

In [13]:
alt.Chart(zipcodes).mark_square(
    size=1, opacity=1
).encode(
    longitude='longitude:Q', # apply the field named 'longitude' to the longitude channel
    latitude='latitude:Q'    # apply the field named 'latitude' to the latitude channel
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

_郵便番号のみをプロットすると、アメリカ合衆国のアウトラインが表示され、基本地図や追加の参照要素なしで郵便局の密度における意味のあるパターンを見分けることができます！_

ここでは`albersUsa`投影法を使用しています。この投影法は、地球の実際の幾何学にいくつかの自由を与え、アラスカとハワイの縮小版を左下隅に配置しています。投影の`scale`や`translate`パラメータを指定しなかったため、Altairは視覚化されたデータに合わせてこれらを自動的に設定します。

これで、データセットに対してさらに質問を投げかけることができます。例えば、郵便番号の割り当てには何か規則性があるのでしょうか？この質問を評価するために、郵便番号の最初の数字に基づく色エンコーディングを追加できます。最初に`calculate`変換を追加して最初の数字を抽出し、その結果を色チャネルでエンコードします：

In [14]:
alt.Chart(zipcodes).transform_calculate(
    digit='datum.zip_code[0]'
).mark_square(
    size=2, opacity=1
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='digit:N'
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

_特定の数字をズームインするには、フィルター変換を追加して表示するデータを制限します！単一の数字にフィルターを絞り、地図を動的に更新するために[インタラクティブな選択](https://github.com/uwdata/visualization-curriculum/blob/master/altair_interaction.ipynb)を追加してみてください。また、フィルターで数字の値を指定する際は文字列（\`'1'\`）を使用してください！_

（この例は、Ben Fryのクラシックな[zipdecode](https://benfry.com/zipdecode/)ビジュアライゼーションにインスパイアされたものです！）

さらに、郵便番号の_順序_が何を示しているのかを考えたくなるかもしれません。この質問を探る一つの方法は、連続する郵便番号を`line`マークを使って接続することです。これはRobert Kosaraの[ZipScribble](https://eagereyes.org/zipscribble-maps/united-states)ビジュアライゼーションで行われた方法です：

In [15]:
alt.Chart(zipcodes).transform_filter(
    '-150 < datum.longitude && 22 < datum.latitude && datum.latitude < 55'
).transform_calculate(
    digit='datum.zip_code[0]'
).mark_line(
    strokeWidth=0.5
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='digit:N',
    order='zip_code:O'
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

_これで、郵便番号がさらに小さなエリアにクラスター化されている様子が見え、場所によるコードの階層的な割り当てが示されていますが、地域内で顕著なばらつきもあります。_

以前の地図に注意深く見ていた場合、左上隅にプロットされた郵便番号があることに気付いたかもしれません！これらはプエルトリコやアメリカ領サモアのような場所に対応しており、`albersUsa`投影法によって`null`座標（`0`, `0`）にマッピングされています。さらに、アラスカとハワイは接続された線分の表示を複雑にする可能性があります。この対応として、上記のコードには選択した`longitude`および`latitude`範囲の外にある点を削除する追加のフィルターが含まれています。

_上記のフィルターを削除して、何が起こるかを見てみてください！_

## シンボル・マップ

今度は、ベースマップとプロットされたデータを別々のレイヤーとして組み合わせます。アメリカの商業航空ネットワークを、空港とフライトルートの両方を考慮して調べます。そのために、3つのデータセットが必要です。
ベースマップには、`states`または`counties`の特徴を含む、解像度10mの米国のTopoJSONファイルを使用します:

In [16]:
usa = data.us_10m.url
usa

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/us-10m.json'

空港については、各空港の`longitude`（経度）と`latitude`（緯度）の座標、そして`iata`空港コード（例：`'SEA'`は[シアトル・タコマ国際空港](https://en.wikipedia.org/wiki/Seattle%E2%80%93Tacoma_International_Airport)を示します）を含むデータセットを使用します。

In [17]:
airports = data.airports.url
airports

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/airports.csv'

最後に、フライトルートのデータセットを使用します。このデータセットには、対応する空港のIATAコードを含む`origin`（出発地）と`destination`（到着地）のフィールドがあります。

In [18]:
flights = data.flights_airport.url
flights

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/flights-airport.csv'

基本地図を作成し、`albersUsa` 投影法を使用して、各空港の `circle` マークをプロットするところから始めましょう。

In [19]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(airports).mark_circle(size=9).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip='iata:N'
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.LayerChart(...)

申し訳ありません。以下が翻訳された全文です。

---

最初に、`albersUsa` 投影法を使ってベースマップを作成し、各空港の位置を `circle` マークで表示するレイヤーを追加します。

_それは多くの空港ですね！もちろん、すべてが主要なハブ空港というわけではありません。_

先ほどの郵便番号データセットと同様に、空港データには本土以外の場所にある点も含まれています。そのため、再び左上隅に点が表示されています。これらの点をフィルタリングしたい場合、まずその位置を知る必要があります。

_上記の地図投影法を `albers` に変更してみてください。これにより、`albersUsa` の特異な挙動を回避し、これらの追加のポイントが実際にどこにあるかが確認できるようになります！_

次に、すべての空港を均等に表示するのではなく、各空港から発着する便の総数を考慮して主要なハブ空港を特定しましょう。`routes` データセットを主なデータソースとして利用します。これには、各 `origin` 空港からの便数を集計したリストが含まれています。

しかし、`routes` データセットには空港の位置情報が含まれていません！そのため、`routes` データに位置情報を追加するために、新しいデータ変換である `lookup` を使用する必要があります。`lookup` 変換は、主データセットのフィールド値をキーとして使用し、別のテーブルから関連情報を検索します。この場合、`routes` データセットの `origin` 空港コードを `airports` データセットの `iata` フィールドと照合し、対応する `latitude` と `longitude` の情報を抽出します。

In [20]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending')
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.LayerChart(...)

_どの米国の空港が最も多くの出発ルートを持っているか？_

空港が表示できたので、空港間の空の交通ネットワークをよりよく理解するために、インタラクティブに操作したいかもしれません。`origin` 空港から `destination` 空港へのルートを示す `rule` マークレイヤーを追加することができます。これには、各エンドポイントの座標を取得するために 2 回の `lookup` 変換が必要です。また、`single` 選択を使用してこれらのルートをフィルタリングし、現在選択された空港から出発するルートのみを表示できるようにします。

_上記の静的な地図から、インタラクティブ版を作成するにはどうすればよいでしょうか？コードの部分をスキップして、インタラクティブな地図に触れ、どのように作成するかを考えてみてください！_

In [21]:
# interactive selection for origin airport
# select nearest airport to mouse cursor
origin = alt.selection_single(
    on='mouseover', nearest=True,
    fields=['origin'], empty='none'
)

# shared data reference for lookup transforms
foreign = alt.LookupData(data=airports, key='iata',
                         fields=['latitude', 'longitude'])
    
alt.layer(
    # base map of the United States
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    # route lines from selected origin airport to destination airports
    alt.Chart(flights).mark_rule(
        color='#000', opacity=0.35
    ).transform_filter(
        origin # filter to selected origin only
    ).transform_lookup(
        lookup='origin', from_=foreign # origin lat/lon
    ).transform_lookup(
        lookup='destination', from_=foreign, as_=['lat2', 'lon2'] # dest lat/lon
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        latitude2='lat2',
        longitude2='lon2',
    ),
    # size airports by number of outgoing routes
    # 1. aggregate flights-airport data set
    # 2. lookup location data from airports data set
    # 3. remove Puerto Rico (PR) and Virgin Islands (VI)
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).add_selection(
        origin
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending') # place smaller circles on top
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

/Users/yuichiyazaki/.pyenv/versions/miniforge3-4.10.3-10/lib/python3.9/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/yuichiyazaki/.pyenv/versions/miniforge3-4.10.3-10/lib/python3.9/site-packages/altair/vegalite/v5/api.py:398: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
/Users/yuichiyazaki/.pyenv/versions/miniforge3-4.10.3-10/lib/python3.9/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.LayerChart(...)

_Mouseover the map to probe the flight network!_

## コロプレス・マップ

[コロプレス・マップ](https://ja.wikipedia.org/wiki/%E3%82%B3%E3%83%AD%E3%83%97%E3%83%AC%E3%82%B9%E5%9B%B3)は、データ値を視覚化するためにシェーディングやテクスチャを使用する地図です。サイズを調整したシンボルマップは、円の面積の比例的な差異を色のシェードで比較するよりも人々がより正確に読み取れるため、しばしば優れています。それにもかかわらず、コロプレス・マップは実際に人気があり、シンボルが多すぎる場合に視覚的に圧倒されることを避けるのに特に有用です。

たとえば、アメリカ合衆国には50の州しかありませんが、その中には数千の郡があります。2008年の不況の年における郡別の失業率のコロプレス・マップを作成してみましょう。場合によっては、入力GeoJSONまたはTopoJSONファイルに、直接視覚化できる統計データが含まれていることがあります。しかしこの場合、2つのファイルがあります。郡の境界特徴（`usa`）が含まれているTopoJSONファイルと、失業統計が含まれている別のテキストファイルです:

In [22]:
unemp = data.unemployment.url
unemp

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/unemployment.tsv'

データソースを統合するために、再度`lookup`変換を使用し、TopoJSONベースの`geoshape`データに失業率を追加する必要があります。その後、`rate`フィールドを参照する`color`エンコーディングを含む地図を作成できます。

In [23]:
alt.Chart(alt.topo_feature(usa, 'counties')).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).encode(
    alt.Color('rate:Q',
              scale=alt.Scale(domain=[0, 0.3], clamp=True), 
              legend=alt.Legend(format='%')),
    alt.Tooltip('rate:Q', format='.0%')
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

*郡ごとの失業率を調べてみましょう。ミシガン州の高い値は自動車産業に関連している可能性があります。グレートプレーンズや山岳地帯の郡では、低い値と高い値の両方が見られます。この変動は意味があるのでしょうか、それとも[サンプルサイズが小さいことによるアーティファクト](https://medium.com/@uwdata/surprise-maps-showing-the-unexpected-e92b67398865)かもしれません。さらに調べるために、色のマッピングを調整するために上限スケールのドメイン（例えば`0.2`）を変更してみてください。*

*コロプレスマップでの中心的な懸念は色の選択です。上記では、Altairのデフォルトの`'yellowgreenblue'`スキームを使用してヒートマップを作成しています。以下では、他のスキームを比較します。例えば、明度だけが変化する**単一色の順次スキーム**（`teals`）、明度と色相の両方が変化する**多色の順次スキーム**（`viridis`）、および白い中間点を使用する**分岐スキーム**（`blueorange`）です。:

In [24]:
# utility function to generate a map specification for a provided color scheme
def map_(scheme):
    return alt.Chart().mark_geoshape().project(type='albersUsa').encode(
        alt.Color('rate:Q', scale=alt.Scale(scheme=scheme), legend=None)
    ).properties(width=305, height=200)

alt.hconcat(
    map_('tealblues'), map_('viridis'), map_('blueorange'),
    data=alt.topo_feature(usa, 'counties')
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).configure_view(
    stroke=None
).resolve_scale(
    color='independent'
)

alt.HConcatChart(...)

_どのカラースキームがより効果的だと感じますか？その理由は何でしょうか？上記のマップを、[Vegaカラースキームのドキュメント](https://vega.github.io/vega/docs/schemes/)に記載されている他の利用可能なスキームを使用するように変更してみてください。_

## Cartographic Projections

これで地図作成に関する経験が得られましたので、カートグラフィック投影法についてもう少し詳しく見ていきましょう。[Wikipedia](https://en.wikipedia.org/wiki/Map_projection)によると、

> _すべての地図投影法は必然的に何らかの形で表面を歪めます。地図の目的に応じて、いくつかの歪みは許容され、他のものは許容されないため、球体のような物体のいくつかの特性を保持するために他の特性を犠牲にする異なる地図投影法が存在します。_

考慮すべきいくつかの特性には次のようなものがあります：

- _面積_: 投影法は地域のサイズを歪めますか？
- _方位_: 直線が一定の進行方向を示すことがありますか？
- _距離_: 等しい長さの線が地球上で等しい距離に対応しますか？
- _形状_: 投影法は点間の空間的関係（角度）を保持しますか？

適切な投影法の選択は、地図の使用目的によって異なります。たとえば、土地利用や土地の広がりが重要な場合、面積を保持する投影法を選ぶかもしれません。地震の震源地から放射される衝撃波を視覚化したい場合、その点からの距離を保持することに焦点を当てるかもしれません。また、航海を支援したい場合、方位や形状の保持が重要になることがあります。

投影法はまた、_投影面_という観点で特徴付けることもできます。たとえば、円筒投影法は球体の表面点を周囲の円筒に投影し、「広げた」円筒が地図として提供されます。次に説明するように、円錐面（円錐投影）や平面（方位投影）に直接投影することもあります。

*まずはさまざまな投影法を操作して直感を養いましょう！**[オンラインVega-Liteカートグラフィック投影ノートブックを開く](https://observablehq.com/@vega/vega-lite-cartographic-projections)。* ページ上のコントロールを使用して、投影法を選択し、`scale`（ズーム）やx/yの平行移動（パン）などの投影パラメータを探ってみてください。回転（[ヨー、ピッチ、ロール](https://en.wikipedia.org/wiki/Aircraft_principal_axes)）コントロールは、地球が投影面に対してどのように向いているかを決定します。

### 特定の投影法の紹介

[**円筒投影法**](https://en.wikipedia.org/wiki/Map_projection#Cylindrical)は、球体を周囲の円筒に投影し、その後円筒を広げて地図にします。円筒の主要軸が南北方向に向いている場合、経線は直線にマッピングされます。

[擬似円筒投影法](https://en.wikipedia.org/wiki/Map_projection#Pseudocylindrical)では、中央経線を直線として表現し、他の経線は中央から「曲がって」いきます。

In [25]:
minimap = map.properties(width=225, height=225)
alt.hconcat(
    minimap.project(type='equirectangular').properties(title='equirectangular'),
    minimap.project(type='mercator').properties(title='mercator'),
    minimap.project(type='transverseMercator').properties(title='transverseMercator'),
    minimap.project(type='naturalEarth1').properties(title='naturalEarth1')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)

- [正距円筒図法](https://en.wikipedia.org/wiki/Equirectangular_projection) (`equirectangular`): 緯度、経度の座標値を直接スケールします。
- [メルカトル図法](https://en.wikipedia.org/wiki/Mercator_projection) (`mercator`): 円筒に投影し、経度はそのまま使用しますが、緯度は非線形変換を受けます。直線は一定の羅針盤の方向（[rhumb lines](https://en.wikipedia.org/wiki/Rhumb_line)）を保持し、航海に適した投影法です。ただし、極地域では面積の歪みが大きくなります。
- [横メルカトル図法](https://en.wikipedia.org/wiki/Transverse_Mercator_projection) (`transverseMercator`): メルカトル図法の一種で、円筒の軸が横向きに回転したものです。標準のメルカトル図法では赤道付近で最も正確ですが、横メルカトル図法では中央子午線に沿った部分で最も正確です。
- [ナチュラルアース図法](https://en.wikipedia.org/wiki/Natural_Earth_projection) (`naturalEarth1`): 地球全体を一度に表示するために設計された擬似円筒図法です。

[**円錐図法**](https://en.wikipedia.org/wiki/Map_projection#Conic)は、球体を円錐に投影し、その円錐を平面に展開します。円錐図法は、球体と円錐が交差する場所を決定する2つの**標準平行線**によって構成されます。

In [26]:
minimap = map.properties(width=180, height=130)
alt.hconcat(
    minimap.project(type='conicEqualArea').properties(title='conicEqualArea'),
    minimap.project(type='conicEquidistant').properties(title='conicEquidistant'),
    minimap.project(type='conicConformal', scale=35, translate=[90,65]).properties(title='conicConformal'),
    minimap.project(type='albers').properties(title='albers'),
    minimap.project(type='albersUsa').properties(title='albersUsa')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)

- [**円錐等面積図法**](https://en.wikipedia.org/wiki/Albers_projection) (`conicEqualArea`): 面積を保持する円錐図法。形状や距離は保持されませんが、標準平行線内ではおおよそ正確です。
- [**円錐等距離図法**](https://en.wikipedia.org/wiki/Equidistant_conic_projection) (`conicEquidistant`): 経線および標準平行線に沿った距離を保持する円錐図法。
- [**円錐等角図法**](https://en.wikipedia.org/wiki/Lambert_conformal_conic_projection) (`conicConformal`): 形状（局所的な角度）を保持する円錐図法。ただし、面積や距離は保持されません。
- [**アルバース図法**](https://en.wikipedia.org/wiki/Albers_projection) (`albers`): アメリカ合衆国の地図作成に最適化された標準平行線を持つ円錐等面積図法の一種。
- [**アルバースUSA図法**](https://en.wikipedia.org/wiki/Albers_projection) (`albersUsa`): アメリカ合衆国の50州のためのハイブリッド投影法。大陸部、アラスカ、ハワイそれぞれの異なるパラメーターで3つのアルバース投影を組み合わせています。

[**方位図法**](https://en.wikipedia.org/wiki/Map_projection#Azimuthal_%28projections_onto_a_plane%29) は、球面を直接平面に投影する図法です。

In [27]:
minimap = map.properties(width=180, height=180)
alt.hconcat(
    minimap.project(type='azimuthalEqualArea').properties(title='azimuthalEqualArea'),
    minimap.project(type='azimuthalEquidistant').properties(title='azimuthalEquidistant'),
    minimap.project(type='orthographic').properties(title='orthographic'),
    minimap.project(type='stereographic').properties(title='stereographic'),
    minimap.project(type='gnomonic').properties(title='gnomonic')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)

- [方位等面投影](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection) (`azimuthalEqualArea`): 地球全体で面積を正確に投影しますが、形状（局所的な角度）は保存しません。
- [方位等距離投影](https://en.wikipedia.org/wiki/Azimuthal_equidistant_projection) (`azimuthalEquidistant`): 投影中心から地球上の他のすべての点への比例距離を保持します。
- [直射投影](https://en.wikipedia.org/wiki/Orthographic_projection_in_cartography) (`orthographic`): 可視の半球を遠くの平面に投影します。外宇宙から見た地球の視点に近いものです。
- [立体投影](https://en.wikipedia.org/wiki/Stereographic_projection) (`stereographic`): 形状は保存しますが、面積や距離は保存しません。
- [球面直投影](https://en.wikipedia.org/wiki/Gnomonic_projection) (`gnomonic`): 球面の表面を接線平面に直接投影します。地球を取り巻く[大円](https://en.wikipedia.org/wiki/Great_circle)が直線として投影され、最短経路を示します。

## コーダ：地理データの整形

上記の例はすべて、vega-datasetsコレクションから取得したデータで、ジオメトリ（TopoJSON）および表形式データ（空港、失業率）を含んでいます。地理的視覚化を始めるための一般的な課題は、タスクに必要なデータを収集することです。多くのデータ提供者が存在し、たとえば[アメリカ合衆国地質調査所（USGS）](https://www.usgs.gov/products/data/all-data)や[米国国勢調査局（U.S. Census Bureau）](https://www.census.gov/data/datasets.html)などのサービスがあります。

多くの場合、すでに地理的な要素を含むデータがあり、追加の測定値やジオメトリが必要です。以下は、データ収集を始めるための一つのワークフローです：

1. [Natural Earth Data](http://www.naturalearthdata.com/downloads/)にアクセスし、興味のある地域と解像度のデータを選択します。対応するzipファイルをダウンロードします。
2. [MapShaper](https://mapshaper.org/)に移動し、ダウンロードしたzipファイルをページにドロップします。データを必要に応じて修正し、生成されたTopoJSONまたはGeoJSONファイルを「エクスポート」します。
3. MapShaperからエクスポートされたデータをAltairで使用するために読み込みます！

もちろん、地理的データを扱うための他にも多くのツール（オープンソースや商用）が存在します。地理データの操作や地図作成については、マイク・ボストックによる[コマンドライン・カートグラフィー](https://medium.com/@mbostock/command-line-cartography-part-1-897aa8f8ca2c)のチュートリアルシリーズを参照してください。

## まとめ

ここまでで、私たちは地図作成のほんの一端に触れただけです。（まさか、1つのノートブックで何世紀もの学びを伝授できると期待していたわけではないですよね？）例えば、私たちは[_カートグラム_](https://en.wikipedia.org/wiki/Cartogram)や[_地形_](https://en.wikipedia.org/wiki/Topography)の伝達—例えば、イムホフの啓発的な著書『[_Cartographic Relief Presentation_](https://books.google.com/books?id=cVy1Ms43fFYC)』—といったトピックには触れていません。しかし、これで、豊かな地理的可視化を作成するための基礎は十分に学びました。さらに学びたい場合は、マクエアレンの著書『[_How Maps Work: Representation, Visualization, and Design_](https://books.google.com/books?id=xhAvN3B0CkUC)』が、データ可視化の視点から地図作成の貴重な概要を提供しています。